In [141]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Asumiendo que es una ocupacion del 80% de camas
INDICE_OCUPACION_CAMAS = 1.25

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

# Proyección de casos INT al 2035

En este cuadernillo se quiere estimar la deminada que atenderá el INT para el 2035. Con esta demanada se calcularán las cantidades de camas necesarias para suplir tal demanda. Para determinar la demanda se utilizarán:

1. Cantidad de casos teóricos por problemas de salud relevantes seleccionados: Estos se calcularon según la incidencia de cada problema de salud y la proyección de la población al 2035.

2. Cantidad de casos atendidos actualmente por cada problema de salud: Estos se obtuvieron al analizar la base de egresos hospitalarios DEIS.

Con ambos datos, se obtendrá cuanto % de atención ha cubierto el INT de los casos teóricos.

In [143]:
# Obtiene los casos de area de influencia
casos_area_de_influencia = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="casos_area_de_influencia_INT"
)

# Preprocesa el diagnostico
casos_area_de_influencia["Diagnostico"] = (
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0]
)

# Preprocesa los diagnosticos agrupados
casos_area_de_influencia["Diagnosticos Contenidos"] = casos_area_de_influencia[
    "Diagnosticos Contenidos"
].str.split(", ")

# Renombra columnas de la poblacion
casos_area_de_influencia = casos_area_de_influencia.rename(columns=COLUMNAS_POBLACION_INTERES)

In [144]:
# Obtiene los diagnosticos relevantes a filtrar de los egresos hospitalarios
DIAGNOSTICOS_RELEVANTES = list(
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0].unique()
)

# Obtiene los diagnosticos agrupados
DIAGNOSTICOS_AGRUPADOS = list(
    casos_area_de_influencia.explode("Diagnosticos Contenidos")["Diagnosticos Contenidos"]
    .dropna()
    .str.strip()
    .unique()
)

# Suma los diagnosticos agrupados a los diagnosticos relevantes
DIAGNOSTICOS_RELEVANTES += DIAGNOSTICOS_AGRUPADOS
DIAGNOSTICOS_RELEVANTES = sorted(DIAGNOSTICOS_RELEVANTES)

In [145]:
# Genera duplas de diagnosticos a reasignar
DIAGNOSTICOS_A_REASIGNAR = casos_area_de_influencia.query("`Diagnosticos Contenidos`.notna()")[
    ["Diagnostico", "Diagnosticos Contenidos"]
]

In [146]:
casos_area_de_influencia = casos_area_de_influencia.set_index("Diagnostico")

## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [147]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/raw/4_resumen_egresos_nacionales/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
)

# Filtra solamente los datos del INT
egresos_torax = egresos_nacionales.query("ESTABLECIMIENTO_SALUD == 112103").copy()

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [148]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_nacionales = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGNOSTICOS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017 and ANO_EGRESO <= 2019")
    .copy()
)

## Reasginación de diagnósticos y agrupación

En este apartado se reasignarán diagnósticos de la base de datos DEIS a un grupo mayor. Por ejemplo,
los siguientes diagnósticos:

- C341 - Tumor maligno del lóbulo superior, bronquio o pulmón
- C342 - Tumor maligno del lóbulo medio, bronquio o pulmón
- C343 - Tumor maligno del lóbulo inferior, bronquio o pulmón
- C780 - Tumor maligno secundario del pulmón
- C782 - Tumor maligno secundario de la pleura
- D381 - Tumor de comportamiento incierto o desconocido de la tráquea, de los bronquios y del pulmón

Todos estos se reasignarán al diagnóstico C34XN (N, para que no se mezclen con los C34X asignados realmente así). Esto, con el fin de poder calcular las estadísticas agrupadas para todos estos diagnósticos (egresos por persona, días de estada).

In [149]:
for row in DIAGNOSTICOS_A_REASIGNAR.itertuples():
    diagnostico_nuevo = row[1]
    diagnosticos_antiguos = row[2]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    egresos_mas_relevantes_nacionales["DIAG1"] = (
        egresos_mas_relevantes_nacionales["DIAG1"]
        .replace(diagnosticos_antiguos, diagnostico_nuevo)
        .copy()
    )

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


Previo a calcular las métricas para cada diagnóstico, es necesario agrupar algunos de ellos. Esto se debe a que ciertas incidencias fueron imposible de encontrar, como por ejemplo con:

- **I052 - Estenosis mitral con insuficiencia**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I051 - Insuficiencia mitral reumática. Sólamente se calculará la cantidad de casos teóricos para la insuficiencia mitral reumática.

- **I080 - Trastornos de las válvulas mitral y aórtica**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I340 - Insuficiencia (de la válvula) mitral y I350 - Estenosis (de la válvula) aórtica.

- **I081 - Trastornos de las válvulas mitral y tricúspide**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I340 - Insuficiencia (de la válvula) mitral y I361 - Insuficiencia no reumática (de la válvula) tricúspide.

In [150]:
# Reasigna I052
egresos_mas_relevantes_nacionales["DIAG1"] = (
    egresos_mas_relevantes_nacionales["DIAG1"].replace({"I052": "I051"}).copy()
)

# Reasigna I080 a I340 y I350 (Estenosis Mitral y Aortica, respectivamente)
egresos_mas_relevantes_nacionales = build_features.assign_diagnosis(
    egresos_mas_relevantes_nacionales, "I080", "I340", "I350"
)

# Reasigna I081 a I340 y I361 (Estenosis Mitral, y Valvulopatia Tricuspide)
egresos_mas_relevantes_nacionales = build_features.assign_diagnosis(
    egresos_mas_relevantes_nacionales, "I081", "I340", "I361"
)

In [151]:
# Filtra solamente los diagnosticos relevantes para el torax
egresos_mas_relevantes_torax = egresos_mas_relevantes_nacionales.query(
    "ESTABLECIMIENTO_SALUD == 112103"
).copy()

In [152]:
# Obtiene el resumen de metricas para el estrato nacional
metricas_diags_relevantes_nacionales = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_nacionales
)

# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_torax
)

## Estimación de casos Hospitalizados utilizando % de Hospitalización del INT

En este caso, se utilizará el % de hospitalización del INT de los pacientes ambulatorios.

In [153]:
PORCENTAJE_HOSPITALIZADOS = pd.read_excel(
    "../data/external/incidencias_y_prevalencias_INT.xlsx",
    usecols=["Diagnostico", "Porcentaje Hospitalizados de Ambulatorios"],
)

PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.drop_duplicates("Diagnostico")

PORCENTAJE_HOSPITALIZADOS["Diagnostico"] = (
    PORCENTAJE_HOSPITALIZADOS["Diagnostico"].str.split(" - ").str[0]
)
PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.set_index("Diagnostico")[
    "Porcentaje Hospitalizados de Ambulatorios"
]

In [154]:
casos_INT_hospitalizados_porcentaje_amb = casos_area_de_influencia[
    COLUMNAS_POBLACION_INTERES.values()
].mul(PORCENTAJE_HOSPITALIZADOS, axis=0)

# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_INT_porcentaje_amb = casos_INT_hospitalizados_porcentaje_amb.mul(
    metricas_diags_relevantes_torax["egresos_por_paciente_agrupado"]["2017-2019"], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_int_porcentaje_amb = egresos_estimados_INT_porcentaje_amb.mul(
    metricas_diags_relevantes_torax["dias_estada_promedio_agrupado"]["2017-2019"], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_int_porcentaje_amb = (
    dias_estada_estimados_int_porcentaje_amb / 365.25
) * INDICE_OCUPACION_CAMAS

# Calcula las camas totales necesarias
camas_totales_int_porcentaje_amb = camas_estimadas_int_porcentaje_amb.sum()

In [155]:
print(camas_totales_int_porcentaje_amb)

2017    181.905330
2018    185.138828
2019    188.894556
2020    192.707102
2021    194.989653
2022    196.436394
2023    197.655938
2024    198.793333
2025    199.876540
2026    200.910924
2027    201.896970
2028    202.834573
2029    203.722047
2030    204.557567
2031    205.416087
2032    206.191686
2033    206.920041
2034    207.603678
2035    208.242877
dtype: float64


## Obtener resumen para MINSAL

In [156]:
resumen_metricas_hosp = metricas_diags_relevantes_torax[
    ["egresos_por_paciente_agrupado", "dias_estada_promedio_agrupado"]
]
resumen_metricas_hosp.columns = [
    "egresos_por_paciente_agrupado_2017_a_2019",
    "dias_estada_promedio_agrupado_2017_a_2019",
]
resumen_casos = casos_INT_hospitalizados_porcentaje_amb[2035]
resumen_casos.name = "casos_hospitalizados_2035"

resumen_egresos = egresos_estimados_INT_porcentaje_amb[2035]
resumen_egresos.name = "egresos_2035"

resumen_dias_estada = dias_estada_estimados_int_porcentaje_amb[2035]
resumen_dias_estada.name = "dias_estada_totales_2035"

resumen_camas = camas_estimadas_int_porcentaje_amb[2035]
resumen_camas.name = "camas_totales_2035"

resumen_total_hosp = (
    resumen_metricas_hosp.merge(
        resumen_casos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_egresos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_dias_estada,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_camas,
        how="left",
        left_index=True,
        right_index=True,
    )
)

ORDEN_COLUMNAS_RESUMEN = [
    "casos_hospitalizados_2035",
    "egresos_por_paciente_agrupado_2017_a_2019",
    "egresos_2035",
    "dias_estada_promedio_agrupado_2017_a_2019",
    "dias_estada_totales_2035",
    "camas_totales_2035",
]

resumen_total_hosp = resumen_total_hosp[ORDEN_COLUMNAS_RESUMEN]

Finalmente, une el resumen de casos teoricos desde el cuadernillo 1.0 con el resumen de este cuadernillo. Esto obtiene la tabla final para MINSAL.

In [157]:
resumen_casos_teoricos = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
)

resumen_casos_teoricos["Diagnostico_codigo"] = (
    resumen_casos_teoricos["Diagnostico"].str.split(" - ").str[0]
)
resumen_casos_teoricos = resumen_casos_teoricos.set_index("Diagnostico_codigo")

resumen_minsal = resumen_casos_teoricos.merge(
    resumen_total_hosp, how="right", left_index=True, right_index=True
)

resumen_minsal = resumen_minsal.reset_index(drop=True).set_index("Diagnostico")

## Guardar archivos

In [158]:
archivos_a_guardar = {
    "metricas_relevantes_INT": metricas_diags_relevantes_torax,
    "casos_hospitalizados_INT": casos_INT_hospitalizados_porcentaje_amb,
    "egresos_estimados_INT": egresos_estimados_INT_porcentaje_amb,
    "dias_estada_estimados_INT": dias_estada_estimados_int_porcentaje_amb,
    "camas_estimadas_desglosadas_INT": camas_estimadas_int_porcentaje_amb,
    "camas_totales_INT": camas_totales_int_porcentaje_amb,
    "resumen_total_hosp_INT": resumen_total_hosp,
    "resumen_MINSAL": resumen_minsal,
}

with pd.ExcelWriter("../data/interim/estimacion_atencion_cerrada_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in archivos_a_guardar.items():
        print(f"Guardando {nombre_hoja}")
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

Guardando metricas_relevantes_INT
Guardando casos_hospitalizados_INT
Guardando egresos_estimados_INT
Guardando dias_estada_estimados_INT
Guardando camas_estimadas_desglosadas_INT
Guardando camas_totales_INT
Guardando resumen_total_hosp_INT
Guardando resumen_MINSAL


In [160]:
import polars as pl

In [161]:
df_torax = pl.read_csv("../data/raw/6_egresos_torax/egresos_procesados_112103.csv")

In [178]:
resumenes = calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(df_torax, 2017, 2019)

In [179]:
resumenes

dias_estada_totales           n_egresos           n_pacientes_distintos  \
                     2017 2018 2019      2017 2018 2019                  2017   
DIAG1                                                                           
A047                   23    0    0         2    0    0                     2   
A084                    0    6    5         0    1    1                     0   
A099                   19   72   49         2    4    4                     2   
A150                  202  329  218        23   31   10                     8   
A151                  193   21    0         5    2    0                     3   
...                   ...  ...  ...       ...  ...  ...                   ...   
Z541                   15   16    0         1    2    0                     1   
Z548                    0    0    2         0    0    1                     0   
Z941                   21    0    0         2    0    0                     2   
Z942                   17    0    0         1    0    0                     1   
Z988                   42    0    0         6    0    0                     6   

                agrupado_entre_2017_2019                        \
      2018 2019                n_egresos n_pacientes_distintos   
DIAG1                                                            
A047     0    0                        2                     2   
A084     1    1                        2                     2   
A099     3    3                       10                     7   
A150    13   10                       64                    29   
A151     2    0                        7                     5   
...    ...  ...                      ...                   ...   
Z541     1    0                        3                     2   
Z548     0    1                        1                     1   
Z941     0    0                        2                     2   
Z942     0    0                        1                     1   
Z988     0    0                        6                     6   

                                                                     
      dias_estada_totales egresos_por_paciente dias_estada_promedio  
DIAG1                                                                
A047                   23             1.000000            11.500000  
A084                   11             1.000000             5.500000  
A099                  140             1.428571            14.000000  
A150                  749             2.206897            11.703125  
A151                  214             1.400000            30.571429  
...                   ...                  ...                  ...  
Z541                   31             1.500000            10.333333  
Z548                    2             1.000000             2.000000  
Z941                   21             1.000000            10.500000  
Z942                   17             1.000000            17.000000  
Z988                   42             1.000000             7.000000  

[599 rows x 14 columns]